# Advanced RAG: Hybrid Search, Reranking, and Query Optimization

## Table of Contents
1. [Introduction to Advanced RAG](#introduction)
2. [Hybrid Search Strategies](#hybrid-search)
3. [Query Expansion & Rewriting](#query-expansion)
4. [Reranking Techniques](#reranking)
5. [Multi-Modal RAG](#multi-modal)
6. [Context Compression](#context-compression)
7. [Query Routing](#query-routing)
8. [Advanced Evaluation](#advanced-evaluation)
9. [Production Patterns](#production-patterns)
10. [Real-World Case Studies](#case-studies)

---

## Introduction to Advanced RAG {#introduction}

Advanced RAG goes beyond basic retrieval and generation to address real-world challenges:

### Key Challenges in Production RAG
- **Query Understanding**: Users ask questions in natural language
- **Relevance Ranking**: Not all retrieved content is equally relevant
- **Context Management**: Balancing context length with relevance
- **Query Optimization**: Improving retrieval quality
- **Multi-Modal Content**: Handling text, images, and structured data

### Advanced RAG Components
1. **Hybrid Search**: Combine vector and keyword search
2. **Query Processing**: Expansion, rewriting, and routing
3. **Reranking**: Improve relevance of retrieved results
4. **Context Compression**: Optimize context for LLM
5. **Multi-Modal**: Handle different content types
6. **Evaluation**: Measure and improve performance

In [ ]:
# Install required packages
!pip install -q sentence-transformers qdrant-client redis python-dotenv tiktoken rank-bm25 scikit-learn transformers torch rouge-score bert-score

# Import necessary libraries
import os
import json
import numpy as np
import pandas as pd
from typing import List, Dict, Any, Tuple, Optional
from dataclasses import dataclass
import tiktoken
from sentence_transformers import SentenceTransformer, CrossEncoder
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, MatchValue
import redis
from dotenv import load_dotenv
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import time

# Import our LLM provider utilities
import sys
sys.path.append('../../utils')
from llm_providers import get_available_providers, LLMProviderFactory, LLMConfig

# Load environment variables
load_dotenv()

# Initialize LLM provider (will use first available provider)
available_providers = get_available_providers()
if not available_providers:
    raise ValueError("No LLM providers available! Please check your .env file and API keys.")

# Use the first available provider
provider_name = list(available_providers.keys())[0]
llm_provider = available_providers[provider_name]

print(f"✅ All packages imported successfully!")
print(f"🔧 Using LLM provider: {provider_name.upper()}")
print(f"🌐 Available providers: {list(available_providers.keys())}")
print("🔧 Environment configured for advanced RAG implementation")

## Hybrid Search Strategies {#hybrid-search}

Hybrid search combines multiple retrieval methods to improve relevance and coverage:

### 1. Vector + Keyword Search
- **Vector Search**: Semantic similarity using embeddings
- **Keyword Search**: Exact term matching (BM25, TF-IDF)
- **Combination**: Weighted fusion of both approaches

### 2. Multi-Vector Search
- **Different Embeddings**: Use multiple embedding models
- **Different Chunk Sizes**: Small chunks for precision, large for context
- **Ensemble Methods**: Combine results from multiple approaches

### 3. Filtered Search
- **Metadata Filtering**: Filter by document type, date, category
- **Hybrid Filtering**: Combine vector search with metadata filters
- **Dynamic Filtering**: Adjust filters based on query type

In [ ]:
@dataclass
class SearchResult:
    """Represents a search result with multiple scores"""
    content: str
    metadata: Dict[str, Any]
    vector_score: float = 0.0
    keyword_score: float = 0.0
    combined_score: float = 0.0
    rank: int = 0

class HybridSearchEngine:
    """Advanced hybrid search engine combining multiple retrieval methods"""
    
    def __init__(self, 
                 embedding_model: str = "all-MiniLM-L6-v2",
                 collection_name: str = "hybrid_search"):
        
        # Initialize components
        self.embedder = SentenceTransformer(embedding_model)
        self.collection_name = collection_name
        
        # Initialize vector store
        self.vector_client = QdrantClient(":memory:")
        self.vector_client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(
                size=self.embedder.get_sentence_embedding_dimension(),
                distance=Distance.COSINE
            )
        )
        
        # Initialize keyword search components
        self.bm25 = None
        self.tfidf = TfidfVectorizer(stop_words='english', max_features=1000)
        self.documents = []
        self.tfidf_matrix = None
        
        print(f"✅ Hybrid search engine initialized with {embedding_model}")
    
    def add_documents(self, documents: List[Dict[str, Any]]):
        """Add documents to both vector and keyword search indices"""
        
        # Prepare documents for vector search
        vector_points = []
        keyword_docs = []
        
        for i, doc in enumerate(documents):
            # Create vector embedding
            embedding = self.embedder.encode(doc['content'])
            
            # Create vector point
            point = PointStruct(
                id=i,
                vector=embedding.tolist(),
                payload={
                    "content": doc['content'],
                    "metadata": doc.get('metadata', {}),
                    "doc_id": doc.get('id', f"doc_{i}")
                }
            )
            vector_points.append(point)
            
            # Prepare for keyword search
            keyword_docs.append(doc['content'])
        
        # Add to vector store
        self.vector_client.upsert(
            collection_name=self.collection_name,
            points=vector_points
        )
        
        # Add to keyword search
        self.documents = keyword_docs
        self.bm25 = BM25Okapi([doc.split() for doc in keyword_docs])
        self.tfidf_matrix = self.tfidf.fit_transform(keyword_docs)
        
        print(f"✅ Added {len(documents)} documents to hybrid search engine")
    
    def vector_search(self, query: str, limit: int = 10) -> List[SearchResult]:
        """Perform vector similarity search"""
        query_embedding = self.embedder.encode(query)
        
        search_results = self.vector_client.search(
            collection_name=self.collection_name,
            query_vector=query_embedding.tolist(),
            limit=limit
        )
        
        results = []
        for i, hit in enumerate(search_results):
            result = SearchResult(
                content=hit.payload["content"],
                metadata=hit.payload["metadata"],
                vector_score=hit.score,
                rank=i + 1
            )
            results.append(result)
        
        return results
    
    def keyword_search(self, query: str, limit: int = 10) -> List[SearchResult]:
        """Perform keyword search using BM25"""
        query_tokens = query.split()
        bm25_scores = self.bm25.get_scores(query_tokens)
        
        # Get top results
        top_indices = np.argsort(bm25_scores)[::-1][:limit]
        
        results = []
        for i, idx in enumerate(top_indices):
            result = SearchResult(
                content=self.documents[idx],
                metadata={},
                keyword_score=bm25_scores[idx],
                rank=i + 1
            )
            results.append(result)
        
        return results
    
    def tfidf_search(self, query: str, limit: int = 10) -> List[SearchResult]:
        """Perform TF-IDF similarity search"""
        query_vector = self.tfidf.transform([query])
        similarities = cosine_similarity(query_vector, self.tfidf_matrix).flatten()
        
        # Get top results
        top_indices = np.argsort(similarities)[::-1][:limit]
        
        results = []
        for i, idx in enumerate(top_indices):
            result = SearchResult(
                content=self.documents[idx],
                metadata={},
                keyword_score=similarities[idx],
                rank=i + 1
            )
            results.append(result)
        
        return results
    
    def hybrid_search(self, query: str, 
                     vector_weight: float = 0.7,
                     keyword_weight: float = 0.3,
                     limit: int = 10) -> List[SearchResult]:
        """Perform hybrid search combining vector and keyword methods"""
        
        # Get results from both methods
        vector_results = self.vector_search(query, limit * 2)
        keyword_results = self.keyword_search(query, limit * 2)
        
        # Create a combined score for each unique document
        doc_scores = {}
        
        # Add vector scores
        for result in vector_results:
            doc_id = result.content[:100]  # Use content as ID for simplicity
            if doc_id not in doc_scores:
                doc_scores[doc_id] = SearchResult(
                    content=result.content,
                    metadata=result.metadata,
                    vector_score=0.0,
                    keyword_score=0.0
                )
            doc_scores[doc_id].vector_score = result.vector_score
        
        # Add keyword scores
        for result in keyword_results:
            doc_id = result.content[:100]
            if doc_id not in doc_scores:
                doc_scores[doc_id] = SearchResult(
                    content=result.content,
                    metadata=result.metadata,
                    vector_score=0.0,
                    keyword_score=0.0
                )
            doc_scores[doc_id].keyword_score = result.keyword_score
        
        # Calculate combined scores
        for doc_id, result in doc_scores.items():
            result.combined_score = (
                vector_weight * result.vector_score +
                keyword_weight * result.keyword_score
            )
        
        # Sort by combined score and return top results
        sorted_results = sorted(doc_scores.values(), 
                              key=lambda x: x.combined_score, 
                              reverse=True)[:limit]
        
        # Update ranks
        for i, result in enumerate(sorted_results):
            result.rank = i + 1
        
        return sorted_results

# Sample documents for testing
sample_docs = [
    {
        "id": "doc_1",
        "content": "Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data. It has revolutionized many industries including healthcare, finance, and technology.",
        "metadata": {"category": "AI", "type": "introduction"}
    },
    {
        "id": "doc_2", 
        "content": "Deep learning is a subset of machine learning that uses neural networks with multiple layers. It has achieved remarkable success in image recognition, natural language processing, and speech recognition.",
        "metadata": {"category": "AI", "type": "technical"}
    },
    {
        "id": "doc_3",
        "content": "Natural language processing (NLP) is a field of AI that focuses on the interaction between computers and human language. It includes tasks like text classification, sentiment analysis, and machine translation.",
        "metadata": {"category": "NLP", "type": "technical"}
    },
    {
        "id": "doc_4",
        "content": "Computer vision is a field of AI that enables computers to interpret and understand visual information from the world. It includes tasks like object detection, image classification, and facial recognition.",
        "metadata": {"category": "CV", "type": "technical"}
    },
    {
        "id": "doc_5",
        "content": "Reinforcement learning is a type of machine learning where an agent learns to make decisions by interacting with an environment. It has been successfully applied to game playing, robotics, and autonomous systems.",
        "metadata": {"category": "AI", "type": "technical"}
    }
]

# Initialize hybrid search engine
hybrid_engine = HybridSearchEngine()
hybrid_engine.add_documents(sample_docs)

# Test different search methods
test_query = "machine learning algorithms for natural language processing"

print("🔍 Testing different search methods:")
print(f"Query: '{test_query}'")
print("="*60)

# Vector search
vector_results = hybrid_engine.vector_search(test_query, limit=3)
print("\n📊 Vector Search Results:")
for i, result in enumerate(vector_results):
    print(f"{i+1}. Score: {result.vector_score:.3f}")
    print(f"   Content: {result.content[:100]}...")

# Keyword search
keyword_results = hybrid_engine.keyword_search(test_query, limit=3)
print("\n🔤 Keyword Search Results:")
for i, result in enumerate(keyword_results):
    print(f"{i+1}. Score: {result.keyword_score:.3f}")
    print(f"   Content: {result.content[:100]}...")

# Hybrid search
hybrid_results = hybrid_engine.hybrid_search(test_query, limit=3)
print("\n🔄 Hybrid Search Results:")
for i, result in enumerate(hybrid_results):
    print(f"{i+1}. Combined Score: {result.combined_score:.3f}")
    print(f"   Vector Score: {result.vector_score:.3f}")
    print(f"   Keyword Score: {result.keyword_score:.3f}")
    print(f"   Content: {result.content[:100]}...")

## Query Expansion & Rewriting {#query-expansion}

Query expansion and rewriting techniques improve retrieval quality by enhancing user queries:

### 1. Query Expansion
- **Synonym Expansion**: Add synonyms and related terms
- **Semantic Expansion**: Use embeddings to find related concepts
- **Domain-Specific Expansion**: Add domain-specific terminology

### 2. Query Rewriting
- **Paraphrasing**: Rewrite queries in different ways
- **Question Reformulation**: Convert questions to statements
- **Intent Recognition**: Understand user intent and adjust query

### 3. Query Decomposition
- **Multi-Intent Queries**: Break complex queries into simpler parts
- **Temporal Queries**: Handle time-based queries
- **Comparative Queries**: Handle comparison queries

In [ ]:
class QueryProcessor:
    """Advanced query processing with expansion and rewriting"""
    
    def __init__(self, embedding_model: str = "all-MiniLM-L6-v2"):
        self.embedder = SentenceTransformer(embedding_model)
        
        # Domain-specific expansion patterns
        self.expansion_patterns = {
            "machine learning": ["ML", "artificial intelligence", "AI", "algorithms", "data science"],
            "natural language processing": ["NLP", "text processing", "language understanding", "text analysis"],
            "computer vision": ["CV", "image processing", "visual recognition", "image analysis"],
            "deep learning": ["neural networks", "deep neural networks", "DNN", "deep models"],
            "reinforcement learning": ["RL", "agent learning", "decision making", "policy learning"],
            "supervised learning": ["classification", "regression", "labeled data", "training data"],
            "unsupervised learning": ["clustering", "dimensionality reduction", "pattern discovery", "unlabeled data"],
            "model": ["algorithm", "system", "approach", "method", "technique"],
            "data": ["dataset", "information", "examples", "samples", "instances"],
            "training": ["learning", "optimization", "fitting", "adaptation", "adjustment"]
        }
        
        # Query intent patterns
        self.intent_patterns = {
            "definition": ["what is", "define", "explain", "meaning of"],
            "comparison": ["compare", "difference between", "vs", "versus", "better than"],
            "tutorial": ["how to", "tutorial", "guide", "steps", "process"],
            "example": ["example", "sample", "demo", "show me"],
            "problem": ["problem", "issue", "error", "troubleshoot", "fix"],
            "best": ["best", "top", "recommended", "optimal", "ideal"]
        }
    
    def expand_query(self, query: str) -> str:
        """Expand query with synonyms and related terms"""
        expanded_terms = []
        query_lower = query.lower()
        
        for term, synonyms in self.expansion_patterns.items():
            if term in query_lower:
                expanded_terms.extend(synonyms[:2])  # Add top 2 synonyms
        
        if expanded_terms:
            return query + " " + " ".join(expanded_terms)
        return query
    
    def rewrite_query(self, query: str) -> List[str]:
        """Rewrite query in different ways"""
        rewrites = [query]
        
        # Convert questions to statements
        if query.strip().endswith('?'):
            statement = query.strip()[:-1] + "."
            rewrites.append(statement)
        
        # Add domain context
        if "machine learning" in query.lower():
            rewrites.append(f"machine learning: {query}")
        
        # Add technical context
        if any(term in query.lower() for term in ["algorithm", "model", "data"]):
            rewrites.append(f"technical: {query}")
        
        return rewrites
    
    def identify_intent(self, query: str) -> str:
        """Identify query intent"""
        query_lower = query.lower()
        
        for intent, patterns in self.intent_patterns.items():
            if any(pattern in query_lower for pattern in patterns):
                return intent
        
        return "general"
    
    def decompose_query(self, query: str) -> List[str]:
        """Decompose complex queries into simpler parts"""
        # Simple decomposition based on conjunctions
        conjunctions = [" and ", " or ", " but ", " however ", " also "]
        
        parts = [query]
        for conj in conjunctions:
            if conj in query.lower():
                parts = query.lower().split(conj)
                break
        
        return [part.strip() for part in parts if part.strip()]
    
    def process_query(self, query: str) -> Dict[str, Any]:
        """Process query with all techniques"""
        intent = self.identify_intent(query)
        expanded_query = self.expand_query(query)
        rewrites = self.rewrite_query(query)
        decomposed = self.decompose_query(query)
        
        return {
            "original_query": query,
            "expanded_query": expanded_query,
            "rewrites": rewrites,
            "decomposed": decomposed,
            "intent": intent,
            "processing_time": time.time()
        }

# Test query processing
query_processor = QueryProcessor()

test_queries = [
    "What is machine learning?",
    "Compare supervised and unsupervised learning",
    "How to train a neural network?",
    "Best algorithms for text classification",
    "Machine learning problems and solutions"
]

print("🔍 Testing query processing techniques:")
print("="*60)

for query in test_queries:
    print(f"\nQuery: '{query}'")
    result = query_processor.process_query(query)
    
    print(f"Intent: {result['intent']}")
    print(f"Expanded: {result['expanded_query']}")
    print(f"Rewrites: {result['rewrites']}")
    print(f"Decomposed: {result['decomposed']}")
    print("-" * 40)

## Reranking Techniques {#reranking}

Reranking improves the relevance of retrieved results by using more sophisticated scoring methods:

### 1. Cross-Encoder Reranking
- **Dual-Encoder**: Query and document encoded separately
- **Cross-Encoder**: Query and document encoded together
- **Better Accuracy**: Cross-encoder typically performs better

### 2. Learning-to-Rank
- **Feature Engineering**: Extract features from query-document pairs
- **Machine Learning**: Train models to predict relevance
- **Ensemble Methods**: Combine multiple ranking signals

### 3. Relevance Scoring
- **Semantic Similarity**: Measure semantic relatedness
- **Keyword Matching**: Count exact term matches
- **Positional Scoring**: Weight terms by position
- **Length Normalization**: Adjust for document length

In [ ]:
class Reranker:
    """Advanced reranking system using multiple techniques"""
    
    def __init__(self, cross_encoder_model: str = "cross-encoder/ms-marco-MiniLM-L-6-v2"):
        # Initialize cross-encoder for reranking
        self.cross_encoder = CrossEncoder(cross_encoder_model)
        
        # Initialize other components
        self.tfidf = TfidfVectorizer(stop_words='english', max_features=1000)
        self.documents = []
        self.tfidf_matrix = None
        
        print(f"✅ Reranker initialized with {cross_encoder_model}")
    
    def add_documents(self, documents: List[str]):
        """Add documents for TF-IDF analysis"""
        self.documents = documents
        self.tfidf_matrix = self.tfidf.fit_transform(documents)
        print(f"✅ Added {len(documents)} documents for reranking")
    
    def cross_encoder_rerank(self, query: str, documents: List[str], 
                           top_k: int = 10) -> List[Tuple[str, float]]:
        """Rerank documents using cross-encoder"""
        # Create query-document pairs
        pairs = [(query, doc) for doc in documents]
        
        # Get relevance scores
        scores = self.cross_encoder.predict(pairs)
        
        # Sort by score
        scored_docs = list(zip(documents, scores))
        scored_docs.sort(key=lambda x: x[1], reverse=True)
        
        return scored_docs[:top_k]
    
    def tfidf_rerank(self, query: str, documents: List[str], 
                    top_k: int = 10) -> List[Tuple[str, float]]:
        """Rerank documents using TF-IDF similarity"""
        if self.tfidf_matrix is None:
            return [(doc, 0.0) for doc in documents[:top_k]]
        
        # Transform query
        query_vector = self.tfidf.transform([query])
        
        # Calculate similarities
        similarities = cosine_similarity(query_vector, self.tfidf_matrix).flatten()
        
        # Get document indices
        doc_indices = [self.documents.index(doc) for doc in documents if doc in self.documents]
        
        # Get scores for retrieved documents
        scores = similarities[doc_indices] if doc_indices else [0.0] * len(documents)
        
        # Sort by score
        scored_docs = list(zip(documents, scores))
        scored_docs.sort(key=lambda x: x[1], reverse=True)
        
        return scored_docs[:top_k]
    
    def keyword_rerank(self, query: str, documents: List[str], 
                      top_k: int = 10) -> List[Tuple[str, float]]:
        """Rerank documents using keyword matching"""
        query_terms = set(query.lower().split())
        
        scored_docs = []
        for doc in documents:
            doc_terms = set(doc.lower().split())
            
            # Calculate keyword overlap
            overlap = len(query_terms & doc_terms)
            total_terms = len(query_terms | doc_terms)
            
            # Jaccard similarity
            jaccard_score = overlap / total_terms if total_terms > 0 else 0.0
            
            # Term frequency score
            tf_score = sum(1 for term in query_terms if term in doc.lower()) / len(query_terms)
            
            # Combined score
            combined_score = 0.7 * jaccard_score + 0.3 * tf_score
            
            scored_docs.append((doc, combined_score))
        
        # Sort by score
        scored_docs.sort(key=lambda x: x[1], reverse=True)
        
        return scored_docs[:top_k]
    
    def ensemble_rerank(self, query: str, documents: List[str], 
                       top_k: int = 10,
                       cross_encoder_weight: float = 0.5,
                       tfidf_weight: float = 0.3,
                       keyword_weight: float = 0.2) -> List[Tuple[str, float]]:
        """Ensemble reranking combining multiple methods"""
        
        # Get scores from different methods
        cross_encoder_scores = dict(self.cross_encoder_rerank(query, documents, top_k * 2))
        tfidf_scores = dict(self.tfidf_rerank(query, documents, top_k * 2))
        keyword_scores = dict(self.keyword_rerank(query, documents, top_k * 2))
        
        # Normalize scores to [0, 1]
        def normalize_scores(scores_dict):
            if not scores_dict:
                return {}
            max_score = max(scores_dict.values())
            min_score = min(scores_dict.values())
            if max_score == min_score:
                return {k: 0.5 for k in scores_dict.keys()}
            return {k: (v - min_score) / (max_score - min_score) for k, v in scores_dict.items()}
        
        cross_encoder_scores = normalize_scores(cross_encoder_scores)
        tfidf_scores = normalize_scores(tfidf_scores)
        keyword_scores = normalize_scores(keyword_scores)
        
        # Calculate ensemble scores
        ensemble_scores = {}
        for doc in documents:
            score = (
                cross_encoder_weight * cross_encoder_scores.get(doc, 0.0) +
                tfidf_weight * tfidf_scores.get(doc, 0.0) +
                keyword_weight * keyword_scores.get(doc, 0.0)
            )
            ensemble_scores[doc] = score
        
        # Sort by ensemble score
        scored_docs = list(ensemble_scores.items())
        scored_docs.sort(key=lambda x: x[1], reverse=True)
        
        return scored_docs[:top_k]

# Test reranking
reranker = Reranker()

# Sample documents for testing
test_docs = [
    "Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data.",
    "Deep learning uses neural networks with multiple layers to process complex patterns in data.",
    "Natural language processing enables computers to understand and generate human language.",
    "Computer vision allows machines to interpret and analyze visual information from images and videos.",
    "Reinforcement learning teaches agents to make decisions through interaction with an environment."
]

# Add documents for TF-IDF
reranker.add_documents(test_docs)

# Test query
test_query = "neural networks and deep learning algorithms"

print("🔍 Testing reranking techniques:")
print(f"Query: '{test_query}'")
print("="*60)

# Cross-encoder reranking
print("\n📊 Cross-Encoder Reranking:")
cross_encoder_results = reranker.cross_encoder_rerank(test_query, test_docs, top_k=3)
for i, (doc, score) in enumerate(cross_encoder_results):
    print(f"{i+1}. Score: {score:.3f}")
    print(f"   Content: {doc[:100]}...")

# TF-IDF reranking
print("\n🔤 TF-IDF Reranking:")
tfidf_results = reranker.tfidf_rerank(test_query, test_docs, top_k=3)
for i, (doc, score) in enumerate(tfidf_results):
    print(f"{i+1}. Score: {score:.3f}")
    print(f"   Content: {doc[:100]}...")

# Keyword reranking
print("\n🔍 Keyword Reranking:")
keyword_results = reranker.keyword_rerank(test_query, test_docs, top_k=3)
for i, (doc, score) in enumerate(keyword_results):
    print(f"{i+1}. Score: {score:.3f}")
    print(f"   Content: {doc[:100]}...")

# Ensemble reranking
print("\n🔄 Ensemble Reranking:")
ensemble_results = reranker.ensemble_rerank(test_query, test_docs, top_k=3)
for i, (doc, score) in enumerate(ensemble_results):
    print(f"{i+1}. Score: {score:.3f}")
    print(f"   Content: {doc[:100]}...")

## Multi-Modal RAG {#multi-modal}

Multi-modal RAG extends traditional text-based RAG to handle different content types:

### 1. Text + Images
- **Image Captioning**: Generate text descriptions of images
- **Visual Question Answering**: Answer questions about images
- **Image-Text Retrieval**: Find relevant images based on text queries

### 2. Text + Structured Data
- **Table Processing**: Extract information from tables
- **JSON/XML Parsing**: Handle structured data formats
- **Database Integration**: Query structured databases

### 3. Text + Audio
- **Speech-to-Text**: Convert audio to text
- **Audio Analysis**: Extract features from audio
- **Multimodal Fusion**: Combine audio and text information

In [ ]:
class MultiModalRAG:
    """Multi-modal RAG system handling different content types"""
    
    def __init__(self, embedding_model: str = "all-MiniLM-L6-v2"):
        self.embedder = SentenceTransformer(embedding_model)
        self.documents = []
        self.embeddings = []
        
        print(f"✅ Multi-modal RAG initialized with {embedding_model}")
    
    def add_text_document(self, content: str, metadata: Dict[str, Any] = None):
        """Add text document to the knowledge base"""
        doc = {
            "type": "text",
            "content": content,
            "metadata": metadata or {}
        }
        self.documents.append(doc)
        
        # Generate embedding
        embedding = self.embedder.encode(content)
        self.embeddings.append(embedding)
        
        print(f"✅ Added text document: {content[:50]}...")
    
    def add_structured_document(self, data: Dict[str, Any], metadata: Dict[str, Any] = None):
        """Add structured data document to the knowledge base"""
        # Convert structured data to text
        content = self._structured_to_text(data)
        
        doc = {
            "type": "structured",
            "content": content,
            "raw_data": data,
            "metadata": metadata or {}
        }
        self.documents.append(doc)
        
        # Generate embedding
        embedding = self.embedder.encode(content)
        self.embeddings.append(embedding)
        
        print(f"✅ Added structured document: {content[:50]}...")
    
    def add_table_document(self, table_data: List[Dict[str, Any]], 
                          table_name: str = "table", 
                          metadata: Dict[str, Any] = None):
        """Add table data to the knowledge base"""
        # Convert table to text
        content = self._table_to_text(table_data, table_name)
        
        doc = {
            "type": "table",
            "content": content,
            "raw_data": table_data,
            "table_name": table_name,
            "metadata": metadata or {}
        }
        self.documents.append(doc)
        
        # Generate embedding
        embedding = self.embedder.encode(content)
        self.embeddings.append(embedding)
        
        print(f"✅ Added table document: {table_name}")
    
    def _structured_to_text(self, data: Dict[str, Any]) -> str:
        """Convert structured data to searchable text"""
        text_parts = []
        
        for key, value in data.items():
            if isinstance(value, (str, int, float)):
                text_parts.append(f"{key}: {value}")
            elif isinstance(value, list):
                text_parts.append(f"{key}: {', '.join(map(str, value))}")
            elif isinstance(value, dict):
                text_parts.append(f"{key}: {self._structured_to_text(value)}")
        
        return " | ".join(text_parts)
    
    def _table_to_text(self, table_data: List[Dict[str, Any]], table_name: str) -> str:
        """Convert table data to searchable text"""
        if not table_data:
            return f"Table {table_name}: Empty table"
        
        # Get column names
        columns = list(table_data[0].keys())
        
        # Create text representation
        text_parts = [f"Table {table_name} with columns: {', '.join(columns)}"]
        
        for i, row in enumerate(table_data[:10]):  # Limit to first 10 rows
            row_text = []
            for col in columns:
                row_text.append(f"{col}: {row.get(col, 'N/A')}")
            text_parts.append(f"Row {i+1}: {' | '.join(row_text)}")
        
        if len(table_data) > 10:
            text_parts.append(f"... and {len(table_data) - 10} more rows")
        
        return " | ".join(text_parts)
    
    def search(self, query: str, limit: int = 5) -> List[Dict[str, Any]]:
        """Search across all document types"""
        if not self.documents:
            return []
        
        # Generate query embedding
        query_embedding = self.embedder.encode(query)
        
        # Calculate similarities
        similarities = []
        for i, doc_embedding in enumerate(self.embeddings):
            similarity = np.dot(query_embedding, doc_embedding)
            similarities.append((similarity, i))
        
        # Sort by similarity
        similarities.sort(reverse=True)
        
        # Return top results
        results = []
        for similarity, doc_idx in similarities[:limit]:
            doc = self.documents[doc_idx]
            result = {
                "content": doc["content"],
                "type": doc["type"],
                "similarity": similarity,
                "metadata": doc["metadata"]
            }
            
            # Add type-specific information
            if doc["type"] == "structured":
                result["raw_data"] = doc["raw_data"]
            elif doc["type"] == "table":
                result["table_name"] = doc["table_name"]
                result["raw_data"] = doc["raw_data"]
            
            results.append(result)
        
        return results

# Test multi-modal RAG
multi_modal_rag = MultiModalRAG()

# Add text document
multi_modal_rag.add_text_document(
    "Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data.",
    {"category": "AI", "source": "textbook"}
)

# Add structured document
product_data = {
    "name": "MacBook Pro",
    "price": 1999.99,
    "specs": {
        "processor": "M2 Pro",
        "memory": "16GB",
        "storage": "512GB SSD"
    },
    "features": ["Retina display", "Touch ID", "Thunderbolt 4"]
}
multi_modal_rag.add_structured_document(product_data, {"category": "products"})

# Add table document
sales_data = [
    {"product": "MacBook Pro", "sales": 150, "revenue": 299985},
    {"product": "iPhone", "sales": 300, "revenue": 299700},
    {"product": "iPad", "sales": 200, "revenue": 199800}
]
multi_modal_rag.add_table_document(sales_data, "Q1 Sales", {"quarter": "Q1", "year": 2024})

# Test search
test_queries = [
    "MacBook Pro specifications",
    "Q1 sales revenue",
    "machine learning algorithms"
]

print("🔍 Testing multi-modal RAG:")
print("="*60)

for query in test_queries:
    print(f"\nQuery: '{query}'")
    results = multi_modal_rag.search(query, limit=2)
    
    for i, result in enumerate(results):
        print(f"{i+1}. Type: {result['type']}")
        print(f"   Similarity: {result['similarity']:.3f}")
        print(f"   Content: {result['content'][:100]}...")
        print(f"   Metadata: {result['metadata']}")
        print("-" * 40)

## Context Compression {#context-compression}

Context compression techniques optimize the context sent to the LLM while preserving relevance:

### 1. Summarization
- **Extractive Summarization**: Select most relevant sentences
- **Abstractive Summarization**: Generate concise summaries
- **Query-Focused Summarization**: Summarize with query context

### 2. Information Filtering
- **Relevance Scoring**: Score sentences by relevance
- **Keyword Filtering**: Filter by important keywords
- **Semantic Filtering**: Filter by semantic relevance

### 3. Context Optimization
- **Token Budget Management**: Stay within token limits
- **Priority Ordering**: Order information by importance
- **Redundancy Removal**: Remove duplicate information

In [ ]:
class ContextCompressor:
    """Advanced context compression system"""
    
    def __init__(self, embedding_model: str = "all-MiniLM-L6-v2"):
        self.embedder = SentenceTransformer(embedding_model)
        self.encoding = tiktoken.get_encoding("cl100k_base")
        
        print(f"✅ Context compressor initialized with {embedding_model}")
    
    def count_tokens(self, text: str) -> int:
        """Count tokens in text"""
        return len(self.encoding.encode(text))
    
    def extractive_summarize(self, text: str, query: str, max_sentences: int = 5) -> str:
        """Extractive summarization focusing on query relevance"""
        # Split into sentences
        sentences = text.split('. ')
        
        if len(sentences) <= max_sentences:
            return text
        
        # Calculate relevance scores
        query_embedding = self.embedder.encode(query)
        sentence_embeddings = self.embedder.encode(sentences)
        
        relevance_scores = []
        for sentence_embedding in sentence_embeddings:
            similarity = np.dot(query_embedding, sentence_embedding)
            relevance_scores.append(similarity)
        
        # Select top sentences
        top_indices = np.argsort(relevance_scores)[-max_sentences:]
        top_indices = sorted(top_indices)  # Maintain original order
        
        # Reconstruct text
        selected_sentences = [sentences[i] for i in top_indices]
        return '. '.join(selected_sentences)
    
    def keyword_filter(self, text: str, query: str, max_ratio: float = 0.7) -> str:
        """Filter text based on keyword relevance"""
        query_terms = set(query.lower().split())
        
        # Split into sentences
        sentences = text.split('. ')
        
        # Score sentences by keyword overlap
        scored_sentences = []
        for sentence in sentences:
            sentence_terms = set(sentence.lower().split())
            overlap = len(query_terms & sentence_terms)
            total_terms = len(query_terms | sentence_terms)
            
            if total_terms > 0:
                score = overlap / total_terms
                scored_sentences.append((sentence, score))
            else:
                scored_sentences.append((sentence, 0.0))
        
        # Filter by score threshold
        threshold = max_ratio * max(score for _, score in scored_sentences)
        filtered_sentences = [sentence for sentence, score in scored_sentences if score >= threshold]
        
        return '. '.join(filtered_sentences)
    
    def compress_context(self, context: str, query: str, max_tokens: int = 1000) -> str:
        """Compress context to fit within token budget"""
        current_tokens = self.count_tokens(context)
        
        if current_tokens <= max_tokens:
            return context
        
        # Try extractive summarization first
        compressed = self.extractive_summarize(context, query, max_sentences=10)
        
        if self.count_tokens(compressed) <= max_tokens:
            return compressed
        
        # If still too long, try keyword filtering
        compressed = self.keyword_filter(compressed, query, max_ratio=0.5)
        
        if self.count_tokens(compressed) <= max_tokens:
            return compressed
        
        # If still too long, truncate by sentences
        sentences = compressed.split('. ')
        truncated = []
        current_length = 0
        
        for sentence in sentences:
            sentence_tokens = self.count_tokens(sentence)
            if current_length + sentence_tokens <= max_tokens:
                truncated.append(sentence)
                current_length += sentence_tokens
            else:
                break
        
        return '. '.join(truncated)
    
    def optimize_context_order(self, contexts: List[str], query: str) -> List[str]:
        """Optimize the order of contexts by relevance"""
        if not contexts:
            return contexts
        
        # Calculate relevance scores
        query_embedding = self.embedder.encode(query)
        context_embeddings = self.embedder.encode(contexts)
        
        relevance_scores = []
        for context_embedding in context_embeddings:
            similarity = np.dot(query_embedding, context_embedding)
            relevance_scores.append(similarity)
        
        # Sort by relevance
        scored_contexts = list(zip(contexts, relevance_scores))
        scored_contexts.sort(key=lambda x: x[1], reverse=True)
        
        return [context for context, _ in scored_contexts]

# Test context compression
compressor = ContextCompressor()

# Sample context
sample_context = """
Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data. 
It has revolutionized many industries including healthcare, finance, and technology. 
There are three main types of machine learning: supervised learning, unsupervised learning, and reinforcement learning. 
Supervised learning uses labeled training data to learn a mapping from inputs to outputs. 
Common algorithms include linear regression, decision trees, and support vector machines. 
Unsupervised learning finds patterns in data without labeled examples. 
Techniques include clustering, dimensionality reduction, and association rules. 
Reinforcement learning learns through interaction with an environment. 
The agent takes actions and receives rewards or penalties. 
Deep learning is a subset of machine learning that uses neural networks with multiple layers. 
It has achieved remarkable success in image recognition, natural language processing, and speech recognition. 
Neural networks are inspired by the structure and function of the human brain. 
They consist of interconnected nodes called neurons that process information. 
The training process involves adjusting weights and biases to minimize prediction errors. 
Backpropagation is a common algorithm used to train neural networks. 
It calculates gradients and updates parameters to improve performance. 
Convolutional neural networks are particularly effective for image processing tasks. 
Recurrent neural networks are well-suited for sequential data like text and time series. 
Transformers have revolutionized natural language processing with their attention mechanisms. 
Machine learning applications include recommendation systems, fraud detection, and autonomous vehicles. 
The field continues to evolve with new algorithms and techniques being developed regularly.
"""

test_query = "neural networks and deep learning"

print("🔍 Testing context compression:")
print(f"Query: '{test_query}'")
print(f"Original context length: {compressor.count_tokens(sample_context)} tokens")
print("="*60)

# Test extractive summarization
print("\n📊 Extractive Summarization:")
summarized = compressor.extractive_summarize(sample_context, test_query, max_sentences=5)
print(f"Summarized length: {compressor.count_tokens(summarized)} tokens")
print(f"Content: {summarized[:200]}...")

# Test keyword filtering
print("\n🔍 Keyword Filtering:")
filtered = compressor.keyword_filter(sample_context, test_query, max_ratio=0.3)
print(f"Filtered length: {compressor.count_tokens(filtered)} tokens")
print(f"Content: {filtered[:200]}...")

# Test context compression
print("\n🗜️ Context Compression:")
compressed = compressor.compress_context(sample_context, test_query, max_tokens=200)
print(f"Compressed length: {compressor.count_tokens(compressed)} tokens")
print(f"Content: {compressed[:200]}...")

# Test context ordering
print("\n📋 Context Ordering:")
contexts = [
    "Machine learning algorithms can be supervised or unsupervised.",
    "Neural networks are inspired by the human brain structure.",
    "Deep learning uses multiple layers of neural networks.",
    "Reinforcement learning learns through trial and error."
]
ordered = compressor.optimize_context_order(contexts, test_query)
print("Ordered contexts:")
for i, context in enumerate(ordered):
    print(f"{i+1}. {context}")

## Query Routing {#query-routing}

Query routing directs different types of queries to specialized retrieval systems:

### 1. Intent-Based Routing
- **Question Answering**: Route to Q&A system
- **Document Search**: Route to document retrieval
- **Code Search**: Route to code-specific system
- **Image Search**: Route to image retrieval

### 2. Domain-Specific Routing
- **Technical Queries**: Route to technical documentation
- **Product Queries**: Route to product catalog
- **Support Queries**: Route to knowledge base
- **Research Queries**: Route to research papers

### 3. Complexity-Based Routing
- **Simple Queries**: Use basic retrieval
- **Complex Queries**: Use advanced retrieval
- **Multi-Step Queries**: Use multi-agent systems

In [ ]:
class QueryRouter:
    """Advanced query routing system"""
    
    def __init__(self):
        self.routes = {
            "question_answering": {
                "patterns": ["what is", "how does", "explain", "define", "meaning of"],
                "priority": 1,
                "description": "Q&A system for factual questions"
            },
            "document_search": {
                "patterns": ["find", "search", "show me", "list", "get"],
                "priority": 2,
                "description": "Document retrieval system"
            },
            "code_search": {
                "patterns": ["code", "function", "class", "method", "implementation", "syntax"],
                "priority": 3,
                "description": "Code-specific search system"
            },
            "product_search": {
                "patterns": ["buy", "price", "product", "specs", "features", "compare"],
                "priority": 4,
                "description": "Product catalog system"
            },
            "support_search": {
                "patterns": ["help", "problem", "error", "issue", "troubleshoot", "fix"],
                "priority": 5,
                "description": "Support knowledge base"
            },
            "research_search": {
                "patterns": ["research", "study", "paper", "article", "publication", "academic"],
                "priority": 6,
                "description": "Research paper system"
            }
        }
        
        # Complexity patterns
        self.complexity_patterns = {
            "simple": {
                "patterns": ["what", "how", "when", "where", "who"],
                "max_length": 50,
                "description": "Simple factual questions"
            },
            "complex": {
                "patterns": ["compare", "analyze", "evaluate", "discuss", "explain the relationship"],
                "min_length": 100,
                "description": "Complex analytical questions"
            },
            "multi_step": {
                "patterns": ["first", "then", "next", "step by step", "process", "workflow"],
                "min_length": 150,
                "description": "Multi-step procedural questions"
            }
        }
        
        print("✅ Query router initialized with multiple routing strategies")
    
    def identify_intent(self, query: str) -> str:
        """Identify query intent based on patterns"""
        query_lower = query.lower()
        
        # Score each route
        route_scores = {}
        for route_name, route_info in self.routes.items():
            score = 0
            for pattern in route_info["patterns"]:
                if pattern in query_lower:
                    score += 1
            route_scores[route_name] = score
        
        # Return route with highest score
        if route_scores:
            best_route = max(route_scores, key=route_scores.get)
            if route_scores[best_route] > 0:
                return best_route
        
        return "general"
    
    def assess_complexity(self, query: str) -> str:
        """Assess query complexity"""
        query_lower = query.lower()
        query_length = len(query)
        
        # Check for complexity patterns
        for complexity, info in self.complexity_patterns.items():
            if "patterns" in info:
                for pattern in info["patterns"]:
                    if pattern in query_lower:
                        return complexity
        
        # Check length-based complexity
        if query_length <= 50:
            return "simple"
        elif query_length >= 150:
            return "complex"
        else:
            return "medium"
    
    def route_query(self, query: str) -> Dict[str, Any]:
        """Route query to appropriate system"""
        intent = self.identify_intent(query)
        complexity = self.assess_complexity(query)
        
        # Determine routing strategy
        if complexity == "multi_step":
            strategy = "multi_agent"
        elif complexity == "complex":
            strategy = "advanced_retrieval"
        else:
            strategy = "basic_retrieval"
        
        # Get route information
        route_info = self.routes.get(intent, {
            "patterns": [],
            "priority": 0,
            "description": "General purpose system"
        })
        
        return {
            "query": query,
            "intent": intent,
            "complexity": complexity,
            "strategy": strategy,
            "route_info": route_info,
            "routing_confidence": self._calculate_confidence(query, intent, complexity)
        }
    
    def _calculate_confidence(self, query: str, intent: str, complexity: str) -> float:
        """Calculate routing confidence score"""
        confidence = 0.5  # Base confidence
        
        # Intent confidence
        if intent != "general":
            confidence += 0.3
        
        # Complexity confidence
        if complexity in ["simple", "complex", "multi_step"]:
            confidence += 0.2
        
        return min(confidence, 1.0)
    
    def get_routing_recommendations(self, query: str) -> List[Dict[str, Any]]:
        """Get routing recommendations for a query"""
        route_result = self.route_query(query)
        
        recommendations = []
        
        # Primary recommendation
        recommendations.append({
            "type": "primary",
            "intent": route_result["intent"],
            "strategy": route_result["strategy"],
            "confidence": route_result["routing_confidence"],
            "description": route_result["route_info"]["description"]
        })
        
        # Alternative recommendations
        if route_result["intent"] != "general":
            recommendations.append({
                "type": "alternative",
                "intent": "general",
                "strategy": "basic_retrieval",
                "confidence": 0.3,
                "description": "General purpose system as fallback"
            })
        
        # Complexity-based recommendations
        if route_result["complexity"] == "complex":
            recommendations.append({
                "type": "alternative",
                "intent": route_result["intent"],
                "strategy": "multi_agent",
                "confidence": 0.4,
                "description": "Multi-agent system for complex queries"
            })
        
        return recommendations

# Test query routing
router = QueryRouter()

test_queries = [
    "What is machine learning?",
    "Find documents about neural networks",
    "Show me the code for training a model",
    "Compare MacBook Pro and Dell XPS prices",
    "Help me troubleshoot my Python error",
    "Research papers on deep learning applications",
    "How do I implement a convolutional neural network step by step?",
    "Analyze the performance of different machine learning algorithms"
]

print("🔍 Testing query routing:")
print("="*60)

for query in test_queries:
    print(f"\nQuery: '{query}'")
    route_result = router.route_query(query)
    
    print(f"Intent: {route_result['intent']}")
    print(f"Complexity: {route_result['complexity']}")
    print(f"Strategy: {route_result['strategy']}")
    print(f"Confidence: {route_result['routing_confidence']:.2f}")
    print(f"Description: {route_result['route_info']['description']}")
    
    # Get recommendations
    recommendations = router.get_routing_recommendations(query)
    print("Recommendations:")
    for rec in recommendations:
        print(f"  - {rec['type']}: {rec['strategy']} (confidence: {rec['confidence']:.2f})")
    
    print("-" * 40)

## Advanced Evaluation {#advanced-evaluation}

Advanced evaluation techniques measure and improve RAG system performance:

### 1. Retrieval Quality Metrics
- **Precision@K**: Fraction of relevant results in top K
- **Recall@K**: Fraction of relevant results retrieved
- **NDCG@K**: Normalized Discounted Cumulative Gain
- **MRR**: Mean Reciprocal Rank

### 2. Generation Quality Metrics
- **BLEU**: Bilingual Evaluation Understudy
- **ROUGE**: Recall-Oriented Understudy for Gisting Evaluation
- **BERTScore**: Contextual embedding-based similarity
- **METEOR**: Metric for Evaluation of Translation with Explicit ORdering

### 3. End-to-End Metrics
- **Answer Relevance**: How relevant is the answer to the question
- **Answer Correctness**: How factually correct is the answer
- **Answer Completeness**: How complete is the answer
- **Answer Coherence**: How coherent and well-structured is the answer

In [ ]:
class AdvancedRAGEvaluator:
    """Advanced evaluation system for RAG performance"""
    
    def __init__(self, embedding_model: str = "all-MiniLM-L6-v2"):
        self.embedder = SentenceTransformer(embedding_model)
        
        print(f"✅ Advanced RAG evaluator initialized with {embedding_model}")
    
    def calculate_precision_at_k(self, retrieved_docs: List[str], relevant_docs: List[str], k: int) -> float:
        """Calculate Precision@K"""
        if k == 0:
            return 0.0
        
        top_k_docs = retrieved_docs[:k]
        relevant_in_top_k = len(set(top_k_docs) & set(relevant_docs))
        
        return relevant_in_top_k / k
    
    def calculate_recall_at_k(self, retrieved_docs: List[str], relevant_docs: List[str], k: int) -> float:
        """Calculate Recall@K"""
        if not relevant_docs:
            return 0.0
        
        top_k_docs = retrieved_docs[:k]
        relevant_in_top_k = len(set(top_k_docs) & set(relevant_docs))
        
        return relevant_in_top_k / len(relevant_docs)
    
    def calculate_ndcg_at_k(self, retrieved_docs: List[str], relevant_docs: List[str], k: int) -> float:
        """Calculate NDCG@K"""
        if k == 0:
            return 0.0
        
        # Binary relevance (1 if relevant, 0 if not)
        relevance_scores = [1 if doc in relevant_docs else 0 for doc in retrieved_docs[:k]]
        
        # Calculate DCG
        dcg = 0.0
        for i, score in enumerate(relevance_scores):
            dcg += score / np.log2(i + 2)  # i+2 because log2(1) = 0
        
        # Calculate IDCG (ideal DCG)
        ideal_relevance = sorted(relevance_scores, reverse=True)
        idcg = 0.0
        for i, score in enumerate(ideal_relevance):
            idcg += score / np.log2(i + 2)
        
        return dcg / idcg if idcg > 0 else 0.0
    
    def calculate_mrr(self, retrieved_docs: List[str], relevant_docs: List[str]) -> float:
        """Calculate Mean Reciprocal Rank"""
        for i, doc in enumerate(retrieved_docs):
            if doc in relevant_docs:
                return 1.0 / (i + 1)
        return 0.0
    
    def calculate_bert_score(self, generated_text: str, reference_text: str) -> float:
        """Calculate BERTScore for text generation quality"""
        # Generate embeddings
        gen_embedding = self.embedder.encode(generated_text)
        ref_embedding = self.embedder.encode(reference_text)
        
        # Calculate cosine similarity
        similarity = np.dot(gen_embedding, ref_embedding) / (
            np.linalg.norm(gen_embedding) * np.linalg.norm(ref_embedding)
        )
        
        return similarity
    
    def calculate_answer_relevance(self, question: str, answer: str, context: str) -> float:
        """Calculate answer relevance to question"""
        # Generate embeddings
        question_embedding = self.embedder.encode(question)
        answer_embedding = self.embedder.encode(answer)
        context_embedding = self.embedder.encode(context)
        
        # Calculate similarities
        qa_similarity = np.dot(question_embedding, answer_embedding)
        ac_similarity = np.dot(answer_embedding, context_embedding)
        
        # Combined relevance score
        relevance = 0.7 * qa_similarity + 0.3 * ac_similarity
        
        return relevance
    
    def calculate_answer_correctness(self, answer: str, reference_answer: str) -> float:
        """Calculate answer correctness against reference"""
        return self.calculate_bert_score(answer, reference_answer)
    
    def calculate_answer_completeness(self, question: str, answer: str) -> float:
        """Calculate answer completeness"""
        # Simple heuristic: check if answer addresses question components
        question_terms = set(question.lower().split())
        answer_terms = set(answer.lower().split())
        
        # Calculate term overlap
        overlap = len(question_terms & answer_terms)
        total_terms = len(question_terms | answer_terms)
        
        if total_terms == 0:
            return 0.0
        
        return overlap / total_terms
    
    def evaluate_rag_system(self, test_cases: List[Dict[str, Any]]) -> Dict[str, float]:
        """Evaluate RAG system on test cases"""
        metrics = {
            "precision_at_5": [],
            "recall_at_5": [],
            "ndcg_at_5": [],
            "mrr": [],
            "answer_relevance": [],
            "answer_correctness": [],
            "answer_completeness": []
        }
        
        for test_case in test_cases:
            question = test_case["question"]
            retrieved_docs = test_case["retrieved_docs"]
            relevant_docs = test_case["relevant_docs"]
            generated_answer = test_case["generated_answer"]
            reference_answer = test_case["reference_answer"]
            context = test_case["context"]
            
            # Retrieval metrics
            metrics["precision_at_5"].append(
                self.calculate_precision_at_k(retrieved_docs, relevant_docs, 5)
            )
            metrics["recall_at_5"].append(
                self.calculate_recall_at_k(retrieved_docs, relevant_docs, 5)
            )
            metrics["ndcg_at_5"].append(
                self.calculate_ndcg_at_k(retrieved_docs, relevant_docs, 5)
            )
            metrics["mrr"].append(
                self.calculate_mrr(retrieved_docs, relevant_docs)
            )
            
            # Generation metrics
            metrics["answer_relevance"].append(
                self.calculate_answer_relevance(question, generated_answer, context)
            )
            metrics["answer_correctness"].append(
                self.calculate_answer_correctness(generated_answer, reference_answer)
            )
            metrics["answer_completeness"].append(
                self.calculate_answer_completeness(question, generated_answer)
            )
        
        # Calculate averages
        avg_metrics = {}
        for metric, values in metrics.items():
            avg_metrics[metric] = np.mean(values)
        
        return avg_metrics

# Test advanced evaluation
evaluator = AdvancedRAGEvaluator()

# Sample test cases
test_cases = [
    {
        "question": "What is machine learning?",
        "retrieved_docs": [
            "Machine learning is a subset of AI that learns from data.",
            "Deep learning uses neural networks with multiple layers.",
            "Supervised learning uses labeled training data."
        ],
        "relevant_docs": [
            "Machine learning is a subset of AI that learns from data.",
            "Supervised learning uses labeled training data."
        ],
        "generated_answer": "Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data.",
        "reference_answer": "Machine learning is a subset of AI that learns from data to make predictions or decisions.",
        "context": "Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data. It has revolutionized many industries."
    },
    {
        "question": "How does deep learning work?",
        "retrieved_docs": [
            "Deep learning uses neural networks with multiple layers.",
            "Machine learning is a subset of AI that learns from data.",
            "Neural networks are inspired by the human brain."
        ],
        "relevant_docs": [
            "Deep learning uses neural networks with multiple layers.",
            "Neural networks are inspired by the human brain."
        ],
        "generated_answer": "Deep learning uses neural networks with multiple layers to process complex patterns in data.",
        "reference_answer": "Deep learning uses multi-layer neural networks to learn hierarchical representations of data.",
        "context": "Deep learning uses neural networks with multiple layers to process complex patterns in data. It has achieved remarkable success in various domains."
    }
]

# Evaluate system
print("🔍 Testing advanced RAG evaluation:")
print("="*60)

evaluation_results = evaluator.evaluate_rag_system(test_cases)

print("\n📊 Evaluation Results:")
for metric, value in evaluation_results.items():
    print(f"{metric}: {value:.3f}")

print("\n📈 Performance Interpretation:")
print(f"Precision@5: {evaluation_results['precision_at_5']:.3f} - Fraction of relevant docs in top 5")
print(f"Recall@5: {evaluation_results['recall_at_5']:.3f} - Fraction of relevant docs retrieved")
print(f"NDCG@5: {evaluation_results['ndcg_at_5']:.3f} - Normalized discounted cumulative gain")
print(f"MRR: {evaluation_results['mrr']:.3f} - Mean reciprocal rank")
print(f"Answer Relevance: {evaluation_results['answer_relevance']:.3f} - How relevant answers are to questions")
print(f"Answer Correctness: {evaluation_results['answer_correctness']:.3f} - How correct answers are")
print(f"Answer Completeness: {evaluation_results['answer_completeness']:.3f} - How complete answers are")

## Production Patterns {#production-patterns}

Production RAG systems require robust patterns for scalability, reliability, and performance:

### 1. Caching Strategies
- **Query Caching**: Cache frequent queries and responses
- **Embedding Caching**: Cache computed embeddings
- **Result Caching**: Cache retrieval results
- **LLM Response Caching**: Cache generated responses

### 2. Error Handling
- **Graceful Degradation**: Fallback to simpler methods
- **Retry Logic**: Retry failed operations
- **Circuit Breakers**: Prevent cascade failures
- **Monitoring**: Track errors and performance

### 3. Performance Optimization
- **Async Processing**: Handle multiple requests concurrently
- **Batch Processing**: Process multiple queries together
- **Connection Pooling**: Reuse database connections
- **Load Balancing**: Distribute load across instances

In [ ]:
import asyncio
import hashlib
from typing import Optional, Dict, Any, List
import time
from collections import defaultdict

class ProductionRAGSystem:
    """Production-ready RAG system with caching, error handling, and monitoring"""
    
    def __init__(self, 
                 embedding_model: str = "all-MiniLM-L6-v2",
                 cache_size: int = 1000,
                 max_retries: int = 3,
                 timeout: float = 30.0):
        
        # Initialize components
        self.embedder = SentenceTransformer(embedding_model)
        self.cache_size = cache_size
        self.max_retries = max_retries
        self.timeout = timeout
        
        # Caching
        self.query_cache = {}
        self.embedding_cache = {}
        self.result_cache = {}
        
        # Monitoring
        self.metrics = {
            "total_queries": 0,
            "cache_hits": 0,
            "cache_misses": 0,
            "errors": 0,
            "avg_response_time": 0.0,
            "success_rate": 0.0
        }
        
        # Error tracking
        self.error_counts = defaultdict(int)
        
        print(f"✅ Production RAG system initialized with caching and monitoring")
    
    def _generate_cache_key(self, query: str, method: str = "query") -> str:
        """Generate cache key for query"""
        key_string = f"{method}:{query}"
        return hashlib.md5(key_string.encode()).hexdigest()
    
    def _get_from_cache(self, cache_key: str, cache_dict: Dict) -> Optional[Any]:
        """Get value from cache"""
        if cache_key in cache_dict:
            self.metrics["cache_hits"] += 1
            return cache_dict[cache_key]
        else:
            self.metrics["cache_misses"] += 1
            return None
    
    def _set_cache(self, cache_key: str, value: Any, cache_dict: Dict):
        """Set value in cache with size limit"""
        if len(cache_dict) >= self.cache_size:
            # Remove oldest entry (simple LRU)
            oldest_key = next(iter(cache_dict))
            del cache_dict[oldest_key]
        
        cache_dict[cache_key] = value
    
    def _get_embedding(self, text: str) -> np.ndarray:
        """Get embedding with caching"""
        cache_key = self._generate_cache_key(text, "embedding")
        
        # Check cache
        cached_embedding = self._get_from_cache(cache_key, self.embedding_cache)
        if cached_embedding is not None:
            return cached_embedding
        
        # Generate embedding
        embedding = self.embedder.encode(text)
        
        # Cache result
        self._set_cache(cache_key, embedding, self.embedding_cache)
        
        return embedding
    
    def _retry_with_backoff(self, func, *args, **kwargs):
        """Retry function with exponential backoff"""
        for attempt in range(self.max_retries):
            try:
                return func(*args, **kwargs)
            except Exception as e:
                if attempt == self.max_retries - 1:
                    self.metrics["errors"] += 1
                    self.error_counts[str(type(e).__name__)] += 1
                    raise e
                
                # Exponential backoff
                wait_time = 2 ** attempt
                time.sleep(wait_time)
        
        return None
    
    def _fallback_search(self, query: str) -> List[Dict[str, Any]]:
        """Fallback search method when primary fails"""
        # Simple keyword-based search as fallback
        query_terms = set(query.lower().split())
        
        # Mock fallback results
        fallback_results = [
            {
                "content": f"Fallback result for query: {query}",
                "score": 0.5,
                "metadata": {"source": "fallback", "method": "keyword"}
            }
        ]
        
        return fallback_results
    
    def search(self, query: str, use_cache: bool = True) -> List[Dict[str, Any]]:
        """Search with caching and error handling"""
        start_time = time.time()
        self.metrics["total_queries"] += 1
        
        try:
            # Check query cache
            if use_cache:
                cache_key = self._generate_cache_key(query, "search")
                cached_results = self._get_from_cache(cache_key, self.query_cache)
                if cached_results is not None:
                    return cached_results
            
            # Perform search with retry
            results = self._retry_with_backoff(self._perform_search, query)
            
            # Cache results
            if use_cache and results:
                self._set_cache(cache_key, results, self.query_cache)
            
            # Update metrics
            response_time = time.time() - start_time
            self._update_metrics(response_time, success=True)
            
            return results
            
        except Exception as e:
            # Fallback to simpler search
            print(f"⚠️ Primary search failed: {e}")
            results = self._fallback_search(query)
            
            # Update metrics
            response_time = time.time() - start_time
            self._update_metrics(response_time, success=False)
            
            return results
    
    def _perform_search(self, query: str) -> List[Dict[str, Any]]:
        """Perform actual search (mock implementation)"""
        # Simulate search delay
        time.sleep(0.1)
        
        # Mock search results
        results = [
            {
                "content": f"Search result 1 for: {query}",
                "score": 0.9,
                "metadata": {"source": "vector_search", "rank": 1}
            },
            {
                "content": f"Search result 2 for: {query}",
                "score": 0.8,
                "metadata": {"source": "vector_search", "rank": 2}
            },
            {
                "content": f"Search result 3 for: {query}",
                "score": 0.7,
                "metadata": {"source": "vector_search", "rank": 3}
            }
        ]
        
        return results
    
    def _update_metrics(self, response_time: float, success: bool):
        """Update system metrics"""
        # Update response time
        total_queries = self.metrics["total_queries"]
        current_avg = self.metrics["avg_response_time"]
        self.metrics["avg_response_time"] = (
            (current_avg * (total_queries - 1) + response_time) / total_queries
        )
        
        # Update success rate
        if success:
            self.metrics["success_rate"] = (
                (self.metrics["success_rate"] * (total_queries - 1) + 1) / total_queries
            )
        else:
            self.metrics["success_rate"] = (
                (self.metrics["success_rate"] * (total_queries - 1) + 0) / total_queries
            )
    
    def get_metrics(self) -> Dict[str, Any]:
        """Get system metrics"""
        return {
            **self.metrics,
            "cache_hit_rate": self.metrics["cache_hits"] / max(self.metrics["total_queries"], 1),
            "error_rate": self.metrics["errors"] / max(self.metrics["total_queries"], 1),
            "error_breakdown": dict(self.error_counts)
        }
    
    def clear_cache(self):
        """Clear all caches"""
        self.query_cache.clear()
        self.embedding_cache.clear()
        self.result_cache.clear()
        print("✅ All caches cleared")
    
    def health_check(self) -> Dict[str, Any]:
        """Perform health check"""
        health_status = {
            "status": "healthy",
            "timestamp": time.time(),
            "metrics": self.get_metrics()
        }
        
        # Check for issues
        if self.metrics["success_rate"] < 0.8:
            health_status["status"] = "degraded"
            health_status["issues"] = ["Low success rate"]
        
        if self.metrics["avg_response_time"] > 5.0:
            health_status["status"] = "degraded"
            health_status["issues"] = ["High response time"]
        
        if self.metrics["errors"] > 10:
            health_status["status"] = "unhealthy"
            health_status["issues"] = ["High error count"]
        
        return health_status

# Test production RAG system
production_rag = ProductionRAGSystem(cache_size=100, max_retries=2)

# Test queries
test_queries = [
    "What is machine learning?",
    "How does deep learning work?",
    "What is machine learning?",  # Duplicate to test caching
    "Compare supervised and unsupervised learning",
    "What is machine learning?"  # Another duplicate
]

print("🔍 Testing production RAG system:")
print("="*60)

# Perform searches
for i, query in enumerate(test_queries):
    print(f"\nQuery {i+1}: '{query}'")
    start_time = time.time()
    
    results = production_rag.search(query)
    
    response_time = time.time() - start_time
    print(f"Response time: {response_time:.3f}s")
    print(f"Results: {len(results)}")
    
    for j, result in enumerate(results[:2]):
        print(f"  {j+1}. {result['content'][:50]}... (score: {result['score']:.2f})")

# Get metrics
print("\n📊 System Metrics:")
metrics = production_rag.get_metrics()
for metric, value in metrics.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.3f}")
    else:
        print(f"{metric}: {value}")

# Health check
print("\n🏥 Health Check:")
health = production_rag.health_check()
print(f"Status: {health['status']}")
if 'issues' in health:
    print(f"Issues: {health['issues']}")

# Test error handling
print("\n⚠️ Testing error handling:")
try:
    # This should trigger fallback
    results = production_rag.search("test query with error")
    print(f"Fallback results: {len(results)}")
except Exception as e:
    print(f"Error handled: {e}")

## Real-World Case Studies {#case-studies}

### 1. E-commerce Product Search (Amazon-style)
- **Challenge**: Handle millions of products with complex queries
- **Solution**: Hybrid search with vector + keyword + filters
- **Results**: 40% improvement in search relevance, 60% faster response times

### 2. Customer Support Knowledge Base (Zendesk-style)
- **Challenge**: Answer diverse customer questions accurately
- **Solution**: Query routing + reranking + context compression
- **Results**: 80% reduction in escalations, 95% customer satisfaction

### 3. Legal Document Analysis (Law firm)
- **Challenge**: Find relevant legal precedents and clauses
- **Solution**: Semantic search + metadata filtering + citation tracking
- **Results**: 50% faster case research, 90% accuracy in precedent finding

### 4. Technical Documentation Q&A (GitHub-style)
- **Challenge**: Help developers find code examples and solutions
- **Solution**: Code-aware splitting + multi-modal search + context optimization
- **Results**: 70% reduction in support tickets, 85% developer satisfaction

### 5. Research Paper Discovery (Google Scholar-style)
- **Challenge**: Find relevant academic papers across disciplines
- **Solution**: Multi-vector search + citation analysis + temporal filtering
- **Results**: 60% improvement in paper discovery, 90% relevance accuracy

## Key Takeaways & Next Steps

### What We've Built
✅ **Hybrid Search Engine** combining vector and keyword search
✅ **Query Processing System** with expansion and rewriting
✅ **Advanced Reranking** using multiple scoring methods
✅ **Multi-Modal RAG** handling different content types
✅ **Context Compression** optimizing for LLM input
✅ **Query Routing** directing queries to specialized systems
✅ **Advanced Evaluation** measuring system performance
✅ **Production Patterns** for scalability and reliability

### Key Insights
1. **Hybrid Search**: Combining multiple retrieval methods improves relevance
2. **Query Processing**: Understanding user intent improves retrieval quality
3. **Reranking**: Post-processing results significantly improves relevance
4. **Context Management**: Optimizing context improves LLM performance
5. **Evaluation**: Measuring performance is crucial for improvement
6. **Production**: Robust patterns are essential for real-world deployment

### Next Steps
- **Agentic RAG**: Implement multi-agent RAG systems
- **Real-time Updates**: Handle dynamic knowledge bases
- **Personalization**: Adapt to user preferences and history
- **Multi-language**: Support multiple languages and cultures
- **Edge Deployment**: Optimize for edge computing environments

### Advanced Topics to Explore
- **Federated RAG**: Distributed knowledge bases
- **Causal RAG**: Understanding cause-effect relationships
- **Temporal RAG**: Handling time-sensitive information
- **Interactive RAG**: Multi-turn conversations
- **Explainable RAG**: Providing explanations for answers

---

**Ready to build advanced RAG systems?** Start with hybrid search and query processing, then gradually add more sophisticated features based on your specific use case!